## Testing RNN

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

## Single Recurrent Neural Network

In [5]:
class SingleRNN(nn.Module):
    def __init__(self, n_inputs, n_neurons):
        super(SingleRNN, self).__init__()
        
        self.Wx = torch.randn(n_inputs, n_neurons) # 4 X 1 Defines input weights
        self.Wy = torch.randn(n_neurons, n_neurons) # 1 X 1 defines weights between time series
        
        self.b = torch.zeros(1, n_neurons) # 1 X 4 Defines biases of inputs
    
    def forward(self, X0, X1):
        self.Y0 = torch.tanh(torch.mm(X0, self.Wx) + self.b) # 4 X 1
        
        self.Y1 = torch.tanh(torch.mm(self.Y0, self.Wy) +
                            torch.mm(X1, self.Wx) + self.b) # 4 X 1
        
        return self.Y0, self.Y1

<img src="RNN_img.png"/>

## Test Model with one neuron

In [3]:
N_INPUT = 4 # Input dim
N_NEURONS = 1

X0_batch = torch.tensor([[0,1,2,0], [3,4,5,0], 
                         [6,7,8,0], [9,0,1,0]],
                        dtype = torch.float) #t=0 => 4 X 4

X1_batch = torch.tensor([[9,8,7,0], [0,0,0,0], 
                         [6,5,4,0], [3,2,1,0]],
                        dtype = torch.float) #t=1 => 4 X 4

model = SingleRNN(N_INPUT, N_NEURONS)

Y0_val, Y1_val = model(X0_batch, X1_batch)

In [4]:
print(Y0_val, Y1_val)

tensor([[-0.9745],
        [-1.0000],
        [-1.0000],
        [-0.8636]]) tensor([[-1.0000],
        [ 0.0091],
        [-1.0000],
        [-0.9296]])


## Model with many neurons

<img src="RNN_img2.png"/>

In [7]:
N_INPUT = 3 # number of features in input
N_NEURONS = 5 # number of units in layer

X0_batch = torch.tensor([[0,1,2], [3,4,5], 
                         [6,7,8], [9,0,1]],
                        dtype = torch.float) #t=0 => 4 X 3

X1_batch = torch.tensor([[9,8,7], [0,0,0], 
                         [6,5,4], [3,2,1]],
                        dtype = torch.float) #t=1 => 4 X 3

model = SingleRNN(N_INPUT, N_NEURONS)

Y0_val, Y1_val = model(X0_batch, X1_batch)

## Write general RNN with N time_steps

In [10]:
class CleanBasicRNN(nn.Module):
    def __init__(self, batch_size, n_inputs, n_neurons, n_time_steps):
        super(CleanBasicRNN, self).__init__()
        
        self.rnn = nn.RNNCell(n_inputs, n_neurons)
        self.hx = torch.randn(batch_size, n_neurons) # initialize hidden state
        self.n_time_steps = n_time_steps
    def forward(self, X):
        output = []

        # for each time step
        for i in range(self.n_time_steps):
            self.hx = self.rnn(X[i], self.hx)
            output.append(self.hx)
        
        return output, self.hx

FIXED_BATCH_SIZE = 4 # our batch size is fixed for now
N_INPUT = 3
N_NEURONS = 5

X_batch = torch.tensor([[[0,1,2], [3,4,5], 
                         [6,7,8], [9,0,1]],
                        [[9,8,7], [0,0,0], 
                         [6,5,4], [3,2,1]]
                       ], dtype = torch.float) # X0 and X1


model = CleanBasicRNN(FIXED_BATCH_SIZE, N_INPUT, N_NEURONS, 2)
output_val, states_val = model(X_batch)
print(output_val) # contains all output for all timesteps
print(states_val) # contains values for final state or final timestep, i.e., t=1

[tensor([[-0.9143, -0.7025,  0.1766, -0.9880, -0.6396],
        [-0.8793,  0.3464, -0.9270, -0.9081,  0.5711],
        [-0.9753,  0.8463, -1.0000, -0.9954,  0.6678],
        [ 0.9631,  0.5100, -0.9504,  0.9551,  0.8417]], grad_fn=<TanhBackward>), tensor([[-0.9099,  0.6987, -1.0000, -0.9713,  0.9791],
        [ 0.2344, -0.1805,  0.2667, -0.5335, -0.4938],
        [-0.7418,  0.7093, -0.9996, -0.9425,  0.6448],
        [-0.6043, -0.6888, -0.5357, -0.6534,  0.0255]], grad_fn=<TanhBackward>)]
tensor([[-0.9099,  0.6987, -1.0000, -0.9713,  0.9791],
        [ 0.2344, -0.1805,  0.2667, -0.5335, -0.4938],
        [-0.7418,  0.7093, -0.9996, -0.9425,  0.6448],
        [-0.6043, -0.6888, -0.5357, -0.6534,  0.0255]], grad_fn=<TanhBackward>)


## Test on MNIST

In [11]:
import torchvision
import torchvision.transforms as transforms

BATCH_SIZE = 64

# list all transformations
transform = transforms.Compose(
    [transforms.ToTensor()])

# download and load training dataset
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

# download and load testing dataset
testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

Processing...
Done!


In [ ]:
# parameters 
N_STEPS = 28
N_INPUTS = 28
N_NEURONS = 150
N_OUTPUTS = 10
N_EPHOCS = 10

## Now we use RNN instead of RNNCell. This allows us to add also hidden layers.

In [ ]:
class ImageRNN(nn.Module):
    def __init__(self, batch_size, n_steps, n_inputs, n_neurons, n_outputs):
        super(ImageRNN, self).__init__()
        
        self.n_neurons = n_neurons
        self.batch_size = batch_size
        self.n_steps = n_steps
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        
        self.basic_rnn = nn.RNN(self.n_inputs, self.n_neurons) 
        
        self.FC = nn.Linear(self.n_neurons, self.n_outputs)
        
    def init_hidden(self,):
        # (num_layers, batch_size, n_neurons)
        return (torch.zeros(1, self.batch_size, self.n_neurons))
        
    def forward(self, X):
        # transforms X to dimensions: n_steps X batch_size X n_inputs
        X = X.permute(1, 0, 2) 
        
        self.batch_size = X.size(1)
        self.hidden = self.init_hidden()
        
        lstm_out, self.hidden = self.basic_rnn(X, self.hidden)      
        out = self.FC(self.hidden)
        
        return out.view(-1, self.n_outputs) # batch_size X n_output